In [50]:
#import BeautifulSoup

import requests
import bs4
from bs4 import BeautifulSoup

In [51]:
# Request from URL
r = requests.get('http://www.indeed.com/jobs?q=data+scientist&l=New+York+State&start=70')
r

<Response [200]>

In [52]:
#print headers and content

print r.headers
print r.content

{'X-Content-Type-Options': 'nosniff', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'CTK=1avdgscf85uah9j8; Domain=.indeed.com; Expires=Sun, 23-Oct-2033 22:37:14 GMT; Path=/, ctkgen=1; Expires=Wed, 19-Oct-2016 03:51:43 GMT; Path=/, JSESSIONID=5E19FD21AEE160955B462A2496A3876B.jasxA_iad-job16; Path=/; HttpOnly, INDEED_CSRF_TOKEN=EW0Mga5zFoG3w4mSbMS8FKhXUewRz1XP; Path=/, PREF=""; Domain=.indeed.com; Expires=Thu, 01-Jan-1970 00:00:10 GMT; Path=/, PREF="TM=1476848923114:L=New+York+State"; Version=1; Max-Age=536870911; Expires=Sun, 23-Oct-2033 22:37:14 GMT; Path=/, RQ="q=data+scientist&l=New+York+State"; Version=1; Max-Age=2592000; Expires=Fri, 18-Nov-2016 03:48:43 GMT; Path=/, UD="LA=1476848923:CV=1476848923:TS=1476848923:SG=61dd151c59b18535f69ded98db57eb8f"; Version=1; Max-Age=536870911; Expires=Sun, 23-Oct-2033 22:37:14 GMT; Path=/, BIGipServerjob_iad=!CkfUDfVdcHEYiM0RNhs+rOzpOInz83Za7Ufqyaf2Wodj8cIIpvgY36gw9dSgmqKBikxM5VIX5GhCM1k=; path=/', 'Vary': 'User-Agent,

In [53]:
# Use BeautifulSoup

insoup = BeautifulSoup(r.content,'lxml')

In [54]:
# function for getting location from indeed

def extract_location_from_result(loc):
    try:
        for n in loc.findAll(class_='location'):
            return n.text.strip()
    except:
        return 'None'

In [55]:
extract_location_from_result(insoup)

u'New York, NY'

In [56]:
# function for getting job title from indeed

def extract_jobtitle_from_result(jobtitle):
    try:
        for n in jobtitle.findAll(class_='jobtitle'):
            return n.text.strip()
    except:
        return 'None' 

In [57]:
extract_jobtitle_from_result(insoup)

u'Machine Learning Quantitative Analyst'

In [58]:
# function for getting company from indeed

def extract_company_from_result(comp):
    try:
        for n in comp.findAll(class_='company'):
            return n.text.strip()
    except:
        return 'None' 

In [59]:
extract_company_from_result(insoup)

u'Bloomberg'

In [60]:
# function to get salary from indeed

def extract_salary_from_result(sal):
    try:
        for n in sal.findAll('td', class_='snip'):
            return n.nobr
    except:
        return 'None'  

In [61]:
extract_salary_from_result(insoup)

In [62]:
# function to get summary/description from indeed

def extract_summary_from_result(desc):
    try:
        for n in desc.findAll('span', class_='summary'):
            return n.text.strip()
    except:
        return 'None' 

In [63]:
extract_summary_from_result(insoup)

u'The Machine Learning Quantitative Analyst will work in Bloomberg\u2019s Enterprise Solutions area and work collaboratively to build a liquidity tool for banks,...'

In [64]:
Location = []
Job_Title = []
Company = []
Salary = []
Description = []

url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100
cities = ['New+York', 'Chicago', 'Boston', 'San+Francisco', 'Los+Angeles', 'Austin', 'Atlanta']
results = []
for city in set(cities):
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city,start)
        ru = requests.get(url)
        indeed = BeautifulSoup(ru.content, "lxml")
        for item in indeed.findAll('div', class_='result'):
            Location.append(extract_location_from_result(item))
            Job_Title.append(extract_jobtitle_from_result(item))
            Company.append(extract_company_from_result(item))
            Salary.append(extract_salary_from_result(item))
            Description.append(extract_summary_from_result(item))

In [65]:
import pandas as pd

In [66]:
df = pd.DataFrame([Location,Job_Title,Company,Salary,Description]).T 

In [67]:
df.columns= ['Location','Job_Title','Company','Salary','Description']

In [73]:
df.head()

,Location,Job_Title,Company,Salary,Description
7,"Austin, TX",Engineering Scientist - GPS Data Analyst,University of Texas at Austin,"<nobr>$6,250 - $10,833 a month</nobr>",Detailed data analyses of GPS observation and ...
12,"Austin, TX",Research Scientist,University of Texas at Austin,"<nobr>$4,584 a month</nobr>",Provide statistical support on research and su...
26,"Austin, TX",Engineering Scientist Associate - R and D Soft...,University of Texas at Austin,"<nobr>$5,400 - $6,500 a month</nobr>",Develop algorithms and processing methodologie...
27,"Austin, TX",Research Engineering/ Scientist Associate II -...,University of Texas at Austin,"<nobr>$4,916 a month</nobr>",Contribute to funded research projects enablin...
38,"Austin, TX 78717",Research Scientist,The University of Texas at Austin,"<nobr>$4,584 a month</nobr>",Provide statistical support on research and su...


In [69]:
df['Salary'].unique()

array([None, <nobr>$6,250 - $10,833 a month</nobr>,
       <nobr>$4,584 a month</nobr>, <nobr>$5,400 - $6,500 a month</nobr>,
       <nobr>$4,916 a month</nobr>, <nobr>$5,259 - $6,941 a month</nobr>,
       <nobr>$5,541 a month</nobr>, <nobr>$4,599 - $6,066 a month</nobr>,
       <nobr>$140,000 - $160,000 a year</nobr>,
       <nobr>$6,667 a month</nobr>, <nobr>$3,520 - $4,600 a month</nobr>,
       <nobr>$4,000 a month</nobr>, <nobr>$5,259 - $8,624 a month</nobr>,
       <nobr>$23.44 - $29.80 an hour</nobr>,
       <nobr>$3,616 - $5,433 a month</nobr>,
       <nobr>$4,500 - $4,900 a month</nobr>,
       <nobr>$120,000 - $150,000 a year</nobr>,
       <nobr>$104,674 - $151,358 a year</nobr>,
       <nobr>$120,000 a year</nobr>,
       <nobr>$80,000 - $110,000 a year</nobr>,
       <nobr>$110,000 a year</nobr>,
       <nobr>$93,257 - $137,114 a year</nobr>,
       <nobr>$105,000 a year</nobr>,
       <nobr>$135,000 - $165,000 a year</nobr>,
       <nobr>$100,000 - $150,000 a year</nobr>

In [74]:
df = df[df.Salary.notnull()]

In [80]:
df['Salary'].dtype

dtype('O')

In [76]:
df

,Location,Job_Title,Company,Salary,Description
7,"Austin, TX",Engineering Scientist - GPS Data Analyst,University of Texas at Austin,"<nobr>$6,250 - $10,833 a month</nobr>",Detailed data analyses of GPS observation and ...
12,"Austin, TX",Research Scientist,University of Texas at Austin,"<nobr>$4,584 a month</nobr>",Provide statistical support on research and su...
26,"Austin, TX",Engineering Scientist Associate - R and D Soft...,University of Texas at Austin,"<nobr>$5,400 - $6,500 a month</nobr>",Develop algorithms and processing methodologie...
27,"Austin, TX",Research Engineering/ Scientist Associate II -...,University of Texas at Austin,"<nobr>$4,916 a month</nobr>",Contribute to funded research projects enablin...
38,"Austin, TX 78717",Research Scientist,The University of Texas at Austin,"<nobr>$4,584 a month</nobr>",Provide statistical support on research and su...
49,"Austin, TX",Senior Statistical Analyst,Health & Human Services Comm,"<nobr>$5,259 - $6,941 a month</nobr>",Creation of data appropriate statistical sampl...
53,"Austin, TX",Research Engineering/ Scientist Associate III ...,University of Texas at Austin,"<nobr>$5,541 a month</nobr>",Contribute to funded research projects enablin...
63,"Austin, TX",Engineering Scientist - GPS and GNSS Data Analyst,University of Texas at Austin,"<nobr>$6,250 - $10,833 a month</nobr>",Provide detailed data analyses of GPS observat...
65,"Austin, TX",Statistical Analyst,Health & Human Services Comm,"<nobr>$4,599 - $6,066 a month</nobr>",Creation of data appropriate statistical sampl...
67,"Austin, TX",Senior Machine Learning Data Scientist,All-In Analytics,"<nobr>$140,000 - $160,000 a year</nobr>","Machine Learning Data Scientist. Forecasting, ..."


In [83]:
df['Salary'] = df['Salary'].apply(lambda x: str(x))

In [84]:
df['Salary'] = df['Salary'].apply(lambda x: x.replace('$',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace(',',''))


In [87]:
df['Salary'] = df['Salary'].apply(lambda x: x.replace('<nobr>',''))
df['Salary'] = df['Salary'].apply(lambda x: x.replace('</nobr>',''))


In [88]:
df

,Location,Job_Title,Company,Salary,Description
7,"Austin, TX",Engineering Scientist - GPS Data Analyst,University of Texas at Austin,6250 - 10833 a month,Detailed data analyses of GPS observation and ...
12,"Austin, TX",Research Scientist,University of Texas at Austin,4584 a month,Provide statistical support on research and su...
26,"Austin, TX",Engineering Scientist Associate - R and D Soft...,University of Texas at Austin,5400 - 6500 a month,Develop algorithms and processing methodologie...
27,"Austin, TX",Research Engineering/ Scientist Associate II -...,University of Texas at Austin,4916 a month,Contribute to funded research projects enablin...
38,"Austin, TX 78717",Research Scientist,The University of Texas at Austin,4584 a month,Provide statistical support on research and su...
49,"Austin, TX",Senior Statistical Analyst,Health & Human Services Comm,5259 - 6941 a month,Creation of data appropriate statistical sampl...
53,"Austin, TX",Research Engineering/ Scientist Associate III ...,University of Texas at Austin,5541 a month,Contribute to funded research projects enablin...
63,"Austin, TX",Engineering Scientist - GPS and GNSS Data Analyst,University of Texas at Austin,6250 - 10833 a month,Provide detailed data analyses of GPS observat...
65,"Austin, TX",Statistical Analyst,Health & Human Services Comm,4599 - 6066 a month,Creation of data appropriate statistical sampl...
67,"Austin, TX",Senior Machine Learning Data Scientist,All-In Analytics,140000 - 160000 a year,"Machine Learning Data Scientist. Forecasting, ..."
